# Imports

In [ ]:
!pip install roboflow
!pip install torchmetrics

In [ ]:
from __future__ import print_function
from __future__ import division

from os.path import join
from roboflow import Roboflow
from torchvision import datasets, models, transforms
from torchmetrics.classification import BinaryAccuracy, BinaryPrecision, BinaryRecall, BinaryF1Score
from torchvision.io import read_image, ImageReadMode
from torchvision.models import resnet152, ResNet152_Weights
from torch.utils.data import Dataset
from tqdm import tqdm

import copy
import matplotlib.pyplot as plt
import os
import pandas as pd
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import numpy as np

# Fine-Tuning

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, scheduler=None, num_epochs=25):
    since = time.time()
    
    metrics = {
        'accuracy': BinaryAccuracy(),
        'precision': BinaryPrecision(),
        'recall': BinaryRecall(),
        'f1': BinaryF1Score(),
    }
    metrics_history = {}
    for metric in metrics.keys():
        metrics_history[metric] = []

    best_model_weights = copy.deepcopy(model.state_dict())
    best_f1 = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            preds_epoch = []
            targets_epoch = []
            
            # Iterate over data
            for X_batch, y_batch in tqdm(dataloaders[phase]):
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(X_batch)
                    loss = criterion(outputs, y_batch)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                preds_epoch.extend(preds.tolist())
                targets_epoch.extend(y_batch.tolist())
                running_loss += loss.item() * X_batch.size(0)
                running_corrects += torch.sum(preds == y_batch.data)

            preds_epoch = torch.Tensor(preds_epoch)
            targets_epoch = torch.Tensor(targets_epoch)
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc_original = running_corrects.double() / len(dataloaders[phase].dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc_original:.4f}')

            for metric_name, metric_obj in metrics.items():
                metric_val = metric_obj(preds_epoch, targets_epoch)
                print(f'{phase} {metric_name}: {metric_val:.4f}')
                if phase == 'val':
                    # save metrics values
                    metrics_history[metric_name].append(metric_val)
                    
            if phase == 'val':
                # deep copy if the model is best
                epoch_f1 = metrics_history['f1'][-1]
                if epoch_f1 > best_f1:
                    best_f1 = epoch_f1
                    best_model_weights = copy.deepcopy(model.state_dict())
        
        if scheduler is not None:
            scheduler.step()
        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val F1: {best_f1:4f}')

    # load best model weights
    model.load_state_dict(best_model_weights)
    return model, metrics_history

In [ ]:
"""When feature extracting, we do not need to compute the gradients
of all the NN parameters except of the last layer ones.
So for efficiency we set the `requires_grad` attribute to False.

By default, this attribute is set to True.

When finetuning, we leave all (or some) of the `required_grad`
parameters set to True.
"""
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:
print("PyTorch Version: ", torch.__version__)
print("Torchvision Version: ", torchvision.__version__)
print()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)

INPUT_SIZE = 224
NUM_CLASSES = 2
BATCH_SIZE = 32
NUM_EPOCHS = 30
feature_extract = True

# Loading a pre-trained image classification model
weights = ResNet152_Weights.DEFAULT
resnet = resnet152(weights=weights)
# setting `requires_grad` parameters (finetuning - True, feature extraction - False)
set_parameter_requires_grad(resnet, feature_extract)
# changing the number of neurons of the output layer
resnet.fc = nn.Linear(resnet.fc.in_features, NUM_CLASSES)
print("Output layer:", resnet.fc)
# Getting the preprocessing necessary for the model
preprocess = weights.transforms()

PyTorch Version:  1.13.1+cu116

Torchvision Version:  0.14.1+cu116



Output layer: Linear(in_features=2048, out_features=2, bias=True)


## Data Loaders and Transforms

In [ ]:
rf = Roboflow(api_key="ZsZB2FnCmcNaslwIlejv")
project = rf.workspace("mydatasets-bqwxe").project("kaggle-kontur")
dataset = project.version(8).download("multiclass")

loading Roboflow workspace...

loading Roboflow project...


Extracting Dataset Version Zip to Kaggle-Kontur-8 in multiclass:: 100%|██████████| 944/944 [00:00<00:00, 985.21it/s] 


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_dir, df, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.df = df
        self.transform = transform
        self.target_transform = target_transform
        
        # save image paths
        img_names = df['filename']
        img_paths = [os.path.join(img_dir, img_name) for img_name in img_names]
        self.df['img_path'] = img_paths

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image = read_image(self.df.iloc[idx]['img_path'], mode=ImageReadMode.RGB)
        label = self.df.iloc[idx][' 1-0']
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
            
        return image, label

In [ ]:
df_train = pd.read_csv("/content/Kaggle-Kontur-8/train/_classes.csv")
df_valid = pd.read_csv("/content/Kaggle-Kontur-8/valid/_classes.csv")

train_loader = CustomDataset('/content/Kaggle-Kontur-8/train', df_train, preprocess)
valid_loader = CustomDataset('/content/Kaggle-Kontur-8/valid', df_valid, preprocess)

dataloaders_dict = {
    'train': torch.utils.data.DataLoader(train_loader, batch_size=BATCH_SIZE, shuffle=True, num_workers=2),
    'val': torch.utils.data.DataLoader(valid_loader, batch_size=BATCH_SIZE, shuffle=True, num_workers=2),
}

Let us check the training set class balance:

In [ ]:
df_train[' 1-0'].value_counts()

0    633
1    237
Name:  1-0, dtype: int64

The classes are slightly imbalanced. Besides accuracy, it's better to check Precision and Recall metrics also.

## Updating the CNN Weights

In [ ]:
def get_updatable_parameters(model, feature_extracting, verbose=True):
    print("Params to learn:")
    if feature_extracting:
        params_to_update = []
        for name,param in model.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                if verbose:
                    print("\t",name)
    else:
        params_to_update = model.parameters()
        for name,param in model.named_parameters():
            if param.requires_grad == True:
                if verbose:
                    print("\t",name)
                
    return params_to_update

In [ ]:
# Send the model to GPU
resnet = resnet.to(device)


""" Gather the parameters to be optimized/updated in this run. If we are
finetuning we will be updating all parameters. However, if we are
doing feature extract method, we will only update the parameters
that we have just initialized, i.e. the parameters with requires_grad is True.
"""
layers_to_unfreeze = [resnet.fc, resnet.avgpool, resnet.layer4]
for layer in layers_to_unfreeze:
    for param in layer.parameters():
        param.requires_grad = True

params_to_update = get_updatable_parameters(resnet, feature_extract)

Params to learn:

	 layer4.0.conv1.weight

	 layer4.0.bn1.weight

	 layer4.0.bn1.bias

	 layer4.0.conv2.weight

	 layer4.0.bn2.weight

	 layer4.0.bn2.bias

	 layer4.0.conv3.weight

	 layer4.0.bn3.weight

	 layer4.0.bn3.bias

	 layer4.0.downsample.0.weight

	 layer4.0.downsample.1.weight

	 layer4.0.downsample.1.bias

	 layer4.1.conv1.weight

	 layer4.1.bn1.weight

	 layer4.1.bn1.bias

	 layer4.1.conv2.weight

	 layer4.1.bn2.weight

	 layer4.1.bn2.bias

	 layer4.1.conv3.weight

	 layer4.1.bn3.weight

	 layer4.1.bn3.bias

	 layer4.2.conv1.weight

	 layer4.2.bn1.weight

	 layer4.2.bn1.bias

	 layer4.2.conv2.weight

	 layer4.2.bn2.weight

	 layer4.2.bn2.bias

	 layer4.2.conv3.weight

	 layer4.2.bn3.weight

	 layer4.2.bn3.bias

	 fc.weight

	 fc.bias


In [ ]:
optimizer = optim.AdamW(params_to_update)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5, verbose=True)
criterion = nn.CrossEntropyLoss()

# Train and evaluate
resnet_ft, metrics_history = train_model(resnet, dataloaders_dict, criterion, optimizer, scheduler, num_epochs=NUM_EPOCHS)

Adjusting learning rate of group 0 to 1.0000e-03.

Epoch 0/29

----------


100%|██████████| 28/28 [00:11<00:00,  2.53it/s]


train Loss: 0.1074 Acc: 0.9632

train accuracy: 0.9632

train precision: 0.9476

train recall: 0.9156

train f1: 0.9313


100%|██████████| 3/3 [00:01<00:00,  2.89it/s]


val Loss: 0.1191 Acc: 0.9706

val accuracy: 0.9706

val precision: 0.9545

val recall: 0.9545

val f1: 0.9545

Adjusting learning rate of group 0 to 1.0000e-03.



Epoch 1/29

----------


...

val Loss: 0.0814 Acc: 0.9706

val accuracy: 0.9706

val precision: 0.9545

val recall: 0.9545

val f1: 0.9545

Adjusting learning rate of group 0 to 3.1250e-05.



Epoch 29/29

----------


100%|██████████| 28/28 [00:09<00:00,  3.00it/s]


train Loss: 0.0002 Acc: 1.0000

train accuracy: 1.0000

train precision: 1.0000

train recall: 1.0000

train f1: 1.0000


100%|██████████| 3/3 [00:01<00:00,  2.75it/s]

val Loss: 0.0807 Acc: 0.9706

val accuracy: 0.9706

val precision: 0.9545

val recall: 0.9545

val f1: 0.9545

Adjusting learning rate of group 0 to 1.5625e-05.



Training complete in 5m 15s

Best val F1: 0.977778


# Inference

Let us detect brown bears and collect the predicted bears in the dataframe

In [ ]:
def classify(model, df, transform, device):
    model.eval()
    file_names = []
    img_paths = []
    preds = []
    for idx, row in df.iterrows():
        # read and transform the image
        img = read_image(row['img_path'], mode=ImageReadMode.RGB)
        img = transform(img)
        img = img.unsqueeze(0).to(device)
        # predict
        with torch.no_grad():
            output = model(img)
            pred = output.data.cpu().numpy().argmax()
        # save outputs
        file_names.append(row['filename'])
        img_paths.append(row['img_path'])
        preds.append(pred)
        
    content = {
        'file_name': file_names,
        'img_path': img_paths,
        'pred': preds,
    }
    return pd.DataFrame(content)

In [ ]:
df_preds = classify(resnet_ft, valid_loader.df, preprocess, device)

In [ ]:
df_preds['pred'].value_counts()

0    45
1    23
Name: pred, dtype: int64

# Saving the model and results

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_filename = f'resnet_ft_{NUM_EPOCHS}ep.pt'
model_path = f'/content/{model_filename}'

torch.save(resnet_ft, model_path)
!cp -i {model_path} /content/drive/MyDrive/kaggle_kontur/{model_filename}

In [ ]:
df_preds.to_csv('/content/df_preds.csv')